In [ ]:
import os, sys, glob
import numpy as np
import pandas as pd

import keras
from keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D,Dense,Dropout,Bidirectional,TimeDistributed,LeakyReLU
from keras.models import Model,Sequential
from keras import optimizers

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
AFFECTIVE_DATA_FOLDER = "/content/drive/My Drive/Affective Datasets/"
AFFECTIVE_ANX_DATA_FOLDER = AFFECTIVE_DATA_FOLDER + 'Anxiety/'
os.chdir(AFFECTIVE_ANX_DATA_FOLDER)
freq = {'EDA': 4, 'BVP': 64, 'TEMP': 4, 'LABEL':700}
params = ['EDA','BVP','TEMP']

dirs = ['POST/','PRE/','TEST01/','TEST02/','TEST03/','TEST04/','TEST05/','TEST06/','TEST07/','TEST08/']

data_cols = ['id', 'file_location', 'startRowIndex', 'endRowIndex', 'label', 'pid'] 
extra_cols = ['anxiety_state','anxiety_before_STAI', 'anxiety_before_CAI','anxiety_after_STAI', 'anxiety_after_CAI', 'anxiety_after_SAES']
cols = data_cols + extra_cols
#extra cols ->anxiety_state (PPT(1), PREP(2), RELAX(3)) , 
#extra cols ->anxiety_before_STAI, anxiety_before_CAI, 
#extra cols ->anxiety_after_STAI, anxiety_after_CAI, anxiety_after_SAES

In [ ]:
 info_df = pd.read_csv(AFFECTIVE_DATA_FOLDER + 'Affective_datasets_info.csv')

In [ ]:
#first timer

In [ ]:
#first timer
info_df['file_key'] = 'value'
info_df['anxiety_state'] = ''
info_df['anxiety_before_STAI'] = -1
info_df['anxiety_before_CAI'] = -1
info_df['anxiety_after_STAI'] = -1
info_df['anxiety_after_CAI'] = -1
info_df['anxiety_after_SAES'] = -1

In [ ]:
for dir in dirs:
  anxiety_state = dir.replace("/", "")
  ratings_before_file = dir + 'Self_Reports/' + anxiety_state + '_beforePPT.xlsx'
  ratings_after_file = dir + 'Self_Reports/' + anxiety_state + '_afterPPT.xlsx'

  if (os.path.exists(ratings_before_file)):
    ratings_before_ppt_df = pd.read_excel(ratings_before_file, index_col=0)  #POST Selfreports->['POST_afterppt', 'POST_beforeppt',
  ratings_after_ppt_df = pd.read_excel(ratings_after_file, index_col=0)  #POST Selfreports->['POST_afterppt', 'POST_beforeppt', 

  data_dir = dir + 'E4/'
  data_directory_contents = os.listdir(data_dir)  #POST E4->['P004', 'P016', 'P020',
  for subjectdir in data_directory_contents:
    subjectdirectory_contents = os.listdir(data_dir+subjectdir+'/') #P04 -> ['HR_RELAX.xlsx','TEMP_PPT.xlsx',
    for fn in subjectdirectory_contents:
      param = (fn.split('_')[0]) #'HR 
      if param in params:
        datafilelocation = data_dir + subjectdir + '/' + fn  #POST/E4/P04/BVP_RELAX.xlsx
        print("DOING-->", datafilelocation)
        data_df = pd.read_excel(datafilelocation, index_col=0)
        #
        idx = info_df.shape[0]
        new_row = {'id': idx, 'file_location': 'Anxiety/' + datafilelocation, 'startRowIndex':0, 'endRowIndex':(data_df.shape[0]-1), 'label': -1, 'pid': 'Anxiety_'+ str(int(subjectdir[1:]))}
        new_row['file_key'] = param
        new_row['anxiety_state'] = anxiety_state
        flagAnxietyLabelFound = False
        if (ratings_before_ppt_df.shape[0]>0):
          before_df = ratings_before_ppt_df[(ratings_before_ppt_df.index == subjectdir)]
          if (before_df.shape[0]>0):
            before_subj_vals = before_df.iloc[0]
            if 'CAI Trait Full Score' in before_df.columns:
              new_row['anxiety_before_CAI'] = before_subj_vals['CAI Trait Full Score']
            if 'STAI Trait Score' in before_df.columns:
              new_row['anxiety_before_STAI'] = before_subj_vals['STAI Trait Score']
            flagAnxietyLabelFound = True

        after_df = ratings_after_ppt_df[(ratings_after_ppt_df.index == subjectdir)]
        if (after_df.shape[0]>0):
          after_subj_vals = after_df.iloc[0]
          if 'CAI State Score' in after_df.columns:
            new_row['anxiety_after_CAI'] = after_subj_vals['CAI State Score']
          if 'STAI State Score' in after_df.columns:  
            new_row['anxiety_after_STAI'] = after_subj_vals['STAI State Score']
          if 'State Anxiety Enthusiasm Score' in after_df.columns:   
            new_row['anxiety_after_SAES'] = after_subj_vals['State Anxiety Enthusiasm Score']
          flagAnxietyLabelFound = True

        if flagAnxietyLabelFound == True:  
          info_df = info_df.append(new_row, ignore_index=True)
        #
        #break
  #break


DOING--> POST/E4/P016/BVP_RELAX.xlsx
DOING--> POST/E4/P016/BVP_PREP.xlsx
DOING--> POST/E4/P016/BVP_PPT.xlsx
DOING--> POST/E4/P020/BVP_PREP.xlsx
DOING--> POST/E4/P073/BVP_PPT.xlsx
DOING--> POST/E4/P073/TEMP_RELAX.xlsx
DOING--> POST/E4/P073/EDA_RELAX.xlsx
DOING--> POST/E4/P073/BVP_RELAX.xlsx
DOING--> POST/E4/P073/EDA_PREP.xlsx
DOING--> POST/E4/P073/EDA_PPT.xlsx
DOING--> POST/E4/P073/TEMP_PPT.xlsx
DOING--> POST/E4/P073/TEMP_PREP.xlsx
DOING--> POST/E4/P042/BVP_RELAX.xlsx
DOING--> POST/E4/P042/BVP_PREP.xlsx
DOING--> POST/E4/P027/BVP_PPT.xlsx
DOING--> POST/E4/P027/BVP_RELAX.xlsx
DOING--> POST/E4/P027/TEMP_RELAX.xlsx
DOING--> POST/E4/P027/BVP_PREP.xlsx
DOING--> POST/E4/P027/TEMP_PREP.xlsx
DOING--> POST/E4/P027/TEMP_PPT.xlsx
DOING--> POST/E4/P004/TEMP_PREP.xlsx
DOING--> POST/E4/P004/TEMP_PPT.xlsx
DOING--> POST/E4/P044/EDA_PPT.xlsx
DOING--> POST/E4/P044/BVP_RELAX.xlsx
DOING--> POST/E4/P044/EDA_PREP.xlsx
DOING--> POST/E4/P044/BVP_PREP.xlsx
DOING--> POST/E4/P044/BVP_PPT.xlsx
DOING--> POST/E4/P044

In [ ]:
info_df

,id,file_location,startRowIndex,endRowIndex,label,pid,file_key,anxiety_state,anxiety_before_STAI,anxiety_before_CAI,anxiety_after_STAI,anxiety_after_CAI,anxiety_after_SAES,application
0,0,Stress1/EDA_2.csv,0,4575,1,Stress1_2,value,NaN,-1,-1,-1.0,-1.0,-1.0,Stress1
1,1,Stress1/BVP_2.csv,0,73215,1,Stress1_2,value,NaN,-1,-1,-1.0,-1.0,-1.0,Stress1
2,2,Stress1/TEMP_2.csv,0,4575,1,Stress1_2,value,NaN,-1,-1,-1.0,-1.0,-1.0,Stress1
3,3,Stress1/EDA_3.csv,0,4559,1,Stress1_3,value,NaN,-1,-1,-1.0,-1.0,-1.0,Stress1
4,4,Stress1/BVP_3.csv,0,72959,1,Stress1_3,value,NaN,-1,-1,-1.0,-1.0,-1.0,Stress1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1755,1755,Anxiety/TEST08/E4/P039/TEMP_PPT.xlsx,0,721,-1,Anxiety_39,TEMP,TEST08,32,43,-1.0,-1.0,52.0,Anxiety
1756,1756,Anxiety/TEST08/E4/P039/EDA_PPT.xlsx,0,721,-1,Anxiety_39,EDA,TEST08,32,43,-1.0,-1.0,52.0,Anxiety
1757,1757,Anxiety/TEST08/E4/P008/EDA_PPT.xlsx,0,973,-1,Anxiety_8,EDA,TEST08,58,39,-1.0,-1.0,47.0,Anxiety
1758,1758,Anxiety/TEST08/E4/P008/BVP_PPT.xlsx,0,15593,-1,Anxiety_8,BVP,TEST08,58,39,-1.0,-1.0,47.0,Anxiety


In [ ]:
info_df['application'] = ''

for index, row in info_df.iterrows():
  app = row['pid'].split('_')[0]
  info_df.loc[index,'application'] = app

In [ ]:
#divide into 3
eda_lst, bvp_lst, temp_lst = [],[],[]
for index, row in info_df.iterrows():
  if ('EDA' in row['file_location']):
    eda_lst.append(index)
  if ('BVP' in row['file_location']):
    bvp_lst.append(index)
  if ('TEMP' in row['file_location']):
    temp_lst.append(index)

In [ ]:
eda_df = info_df.loc[eda_lst]
bvp_df = info_df.loc[bvp_lst]
temp_df = info_df.loc[temp_lst]

In [ ]:
eda_df.reset_index(drop=True, inplace=True)
bvp_df.reset_index(drop=True, inplace=True)
temp_df.reset_index(drop=True, inplace=True)

In [ ]:
eda_df['id'] = eda_df.index
bvp_df['id'] = bvp_df.index
temp_df['id'] = temp_df.index

In [ ]:
eda_df.to_csv(AFFECTIVE_DATA_FOLDER + 'EDA_info.csv', index = False) 
bvp_df.to_csv(AFFECTIVE_DATA_FOLDER + 'BVP_info.csv', index = False) 
temp_df.to_csv(AFFECTIVE_DATA_FOLDER + 'TEMP_info.csv', index = False) 

In [ ]:
#####

In [ ]:
info_df.anxiety_after_STAI.unique()
info_df['anxiety_after_STAI'] = info_df['anxiety_after_STAI'].fillna(-1)
info_df['anxiety_after_CAI'] = info_df['anxiety_after_CAI'].fillna(-1)
info_df['anxiety_after_SAES'] = info_df['anxiety_after_SAES'].fillna(-1)
info_df['anxiety_before_STAI'] = info_df['anxiety_before_STAI'].fillna(-1)
info_df['anxiety_before_CAI'] = info_df['anxiety_before_CAI'].fillna(-1)

In [ ]:
info_df.anxiety_after_STAI.unique()

array([-1., 20., 22., 21., 35., 70., 23., 38., 28., 36., 43., 30., 40.,
       47., 32., 41., 44., 42., 27., 49., 52., 26., 56., 51., 53., 31.,
       46., 48., 33., 65., 50., 54., 45., 39., 60.])

In [ ]:
info_df.to_csv(AFFECTIVE_DATA_FOLDER + 'Affective_datasets_info.csv', index = False) 

In [ ]:
##end

In [ ]:
idx = info_df.shape[0]
new_row = {'id': idx, 'file_location': 'Anxiety/' + datafilelocation, 'startRowIndex':0, 'endRowIndex':(data_df.shape[0]-1), 'label': -1, 'pid': 'Anxiety_'+ str(int(subjectdir[1:]))}
new_row['file_key'] = param
new_row['anxiety_state'] = anxiety_state

In [ ]:
idx = info_df.shape[0]
new_row = {'id': idx, 'file_location': 'Anxiety/' + datafilelocation, 'startRowIndex':0, 'endRowIndex':(data_df.shape[0]-1), 'label': -1, 'pid': 'Anxiety_'+ str(int(subjectdir[1:]))}
new_row['file_key'] = param
new_row['anxiety_state'] = anxiety_state
if (ratings_before_ppt_df.shape[0]>0):
  before_subj_vals = ratings_before_ppt_df[(ratings_before_ppt_df.index == subjectdir)].iloc[0]
  new_row['anxiety_before_CAI'] = before_subj_vals['CAI Trait Full Score']
  new_row['anxiety_before_STAI'] = before_subj_vals['STAI Trait Score']

after_subj_vals = ratings_after_ppt_df[(ratings_after_ppt_df.index == subjectdir)].iloc[0]
new_row['anxiety_after_CAI'] = after_subj_vals['CAI State Score']
new_row['anxiety_after_STAI'] = after_subj_vals['STAI State Score']
new_row['anxiety_after_SAES'] = after_subj_vals['State Anxiety Enthusiasm Score']
info_df = info_df.append(new_row, ignore_index=True)

In [ ]:
new_row

{'anxiety_after_CAI': 39.0,
 'anxiety_after_SAES': 52.0,
 'anxiety_after_STAI': 41.0,
 'anxiety_before_CAI': 34,
 'anxiety_before_STAI': 49,
 'anxiety_state': 'POST',
 'endRowIndex': 15789,
 'file_key': 'BVP',
 'file_location': 'Anxiety/POST/E4/P008/BVP_PPT.xlsx',
 'id': 645,
 'label': -1,
 'pid': 'Anxiety_8',
 'startRowIndex': 0}

In [ ]:
info_df = info_df.append(new_row, ignore_index=True)

In [ ]:
info_df

,id,file_location,startRowIndex,endRowIndex,label,pid,file_key,anxiety_state,anxiety_before_STAI,anxiety_before_CAI,anxiety_after_STAI,anxiety_after_CAI,anxiety_after_SAES
0,0,Stress1/EDA_2.csv,0,4575,1,Stress1_2,value,,-1,-1,-1.0,-1.0,-1.0
1,1,Stress1/BVP_2.csv,0,73215,1,Stress1_2,value,,-1,-1,-1.0,-1.0,-1.0
2,2,Stress1/TEMP_2.csv,0,4575,1,Stress1_2,value,,-1,-1,-1.0,-1.0,-1.0
3,3,Stress1/EDA_3.csv,0,4559,1,Stress1_3,value,,-1,-1,-1.0,-1.0,-1.0
4,4,Stress1/BVP_3.csv,0,72959,1,Stress1_3,value,,-1,-1,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1755,1755,Anxiety/TEST08/E4/P039/TEMP_PPT.xlsx,0,721,-1,Anxiety_39,TEMP,TEST08,32,43,-1.0,-1.0,52.0
1756,1756,Anxiety/TEST08/E4/P039/EDA_PPT.xlsx,0,721,-1,Anxiety_39,EDA,TEST08,32,43,-1.0,-1.0,52.0
1757,1757,Anxiety/TEST08/E4/P008/EDA_PPT.xlsx,0,973,-1,Anxiety_8,EDA,TEST08,58,39,-1.0,-1.0,47.0
1758,1758,Anxiety/TEST08/E4/P008/BVP_PPT.xlsx,0,15593,-1,Anxiety_8,BVP,TEST08,58,39,-1.0,-1.0,47.0


In [ ]:
###STATTS FIRSTTIME

In [ ]:
 info_df = pd.read_csv(AFFECTIVE_DATA_FOLDER + 'Affective_datasets_info.csv')

In [ ]:
info_df['file_key'] = 'value'
info_df['anxiety_state'] = ''
info_df['anxiety_before_STAI'] = -1
info_df['anxiety_before_CAI'] = -1
info_df['anxiety_after_STAI'] = -1
info_df['anxiety_after_CAI'] = -1
info_df['anxiety_after_SAES'] = -1

In [ ]:
info_df

,id,file_location,startRowIndex,endRowIndex,label,pid,file_key,anxiety_state,anxiety_before_STAI,anxiety_before_CAI,anxiety_after_STAI,anxiety_after_CAI,anxiety_after_SAES
0,0,Stress1/EDA_2.csv,0,4575,1,Stress1_2,value,,-1,-1,-1,-1,-1
1,1,Stress1/BVP_2.csv,0,73215,1,Stress1_2,value,,-1,-1,-1,-1,-1
2,2,Stress1/TEMP_2.csv,0,4575,1,Stress1_2,value,,-1,-1,-1,-1,-1
3,3,Stress1/EDA_3.csv,0,4559,1,Stress1_3,value,,-1,-1,-1,-1,-1
4,4,Stress1/BVP_3.csv,0,72959,1,Stress1_3,value,,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,640,Emotion/TEMP_raw.csv,75132,75211,1,Emotion_31,value,,-1,-1,-1,-1,-1
641,641,Emotion/TEMP_raw.csv,75212,75251,0,Emotion_31,value,,-1,-1,-1,-1,-1
642,642,Emotion/TEMP_raw.csv,75252,75291,1,Emotion_31,value,,-1,-1,-1,-1,-1
643,643,Emotion/TEMP_raw.csv,75292,75373,0,Emotion_31,value,,-1,-1,-1,-1,-1


In [ ]:
info_df

,id,file_location,startRowIndex,endRowIndex,label,pid,file_key,anxiety_state,anxiety_before_STAI,anxiety_before_CAI,anxiety_after_STAI,anxiety_after_CAI,anxiety_after_SAES
0,0,Stress1/EDA_2.csv,0,4575,1,Stress1_2,value,,-1,-1,-1.0,-1.0,-1.0
1,1,Stress1/BVP_2.csv,0,73215,1,Stress1_2,value,,-1,-1,-1.0,-1.0,-1.0
2,2,Stress1/TEMP_2.csv,0,4575,1,Stress1_2,value,,-1,-1,-1.0,-1.0,-1.0
3,3,Stress1/EDA_3.csv,0,4559,1,Stress1_3,value,,-1,-1,-1.0,-1.0,-1.0
4,4,Stress1/BVP_3.csv,0,72959,1,Stress1_3,value,,-1,-1,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,641,Emotion/TEMP_raw.csv,75212,75251,0,Emotion_31,value,,-1,-1,-1.0,-1.0,-1.0
642,642,Emotion/TEMP_raw.csv,75252,75291,1,Emotion_31,value,,-1,-1,-1.0,-1.0,-1.0
643,643,Emotion/TEMP_raw.csv,75292,75373,0,Emotion_31,value,,-1,-1,-1.0,-1.0,-1.0
644,644,Emotion/TEMP_raw.csv,75374,80615,0,Emotion_32,value,,-1,-1,-1.0,-1.0,-1.0


In [ ]:
###ENDFIRSTTIME

In [ ]:
data_df = pd.read_excel(datafilelocation, index_col=0)

In [ ]:
data_df

,BVP
Time (s),
0.011375,14.39
0.027000,16.98
0.042625,17.77
0.058250,17.48
0.073875,16.69
...,...
246.652000,-13.91
246.667625,-11.87
246.683250,-9.05


In [ ]:
data_df

""
HR
73.48
73.37
73.23
73.08
72.92
...
81.87
81.97
82.07


In [ ]:
subdirectory_contents

['BVP_RELAX.xlsx',
 'BVP_PREP.xlsx',
 'BVP_PPT.xlsx',
 'HR_PREP.xlsx',
 'HR_PPT.xlsx',
 'HR_RELAX.xlsx',
 'IBI_RELAX.xlsx',
 'ACC_PREP.xlsx',
 'IBI_PREP.xlsx']

In [ ]:
for fn in subdirectory_contents:
  filelocation = data_dir + subdir + '/' + fn #POST/E4/P016/BVP_RELAX.xlsx
  break

In [ ]:
filelocation

'POST/E4/P016/BVP_RELAX.xlsx'

In [ ]:
####

In [ ]:

for file in glob.glob("*.csv"):
  if file.startswith('S') and 'acc' not in file:
    fn = (file.split('.')[0]).split('_')
    param, subjectId = fn[1], int(fn[0][1:])
    param_df = pd.read_csv(AFFECTIVE_MW_DATA_FOLDER + file)
    break
    #merge labels
    #create info file

In [ ]:
####

In [ ]:
labels_df = pd.read_csv(AFFECTIVE_MW_DATA_FOLDER + "Labels.csv")

In [ ]:
subject_df = labels_df[(labels_df['Participant ID'] == (subjectId+1000))]

In [ ]:
info_df = pd.DataFrame(columns=data_cols)

In [ ]:
idx = 0
new_row = {'id':'Mw_'+ str(idx), 'file_location': file, 'startTime':-1, 'endTime':-1, 'label': int(subject_df["mw_labels"].mean() >= 0.5), 'pid': 'Mw_'+ str(subjectId)}

In [ ]:
info_df = info_df.append(new_row, ignore_index=True)
info_df

,id,file_location,startTime,endTime,label,pid
0,Mw_0,S02_bvp.csv,-1,-1,1,Mw_2


In [ ]:
int(subject_df["mw_labels"].mean() >= 0.5)

1

In [ ]:
####

In [ ]:
labels_df.columns

Index(['Unnamed: 0', 'Participant ID', 'Mental Demand', 'Physical Demand',
       'Temporal Demand', 'Performance', 'Effort', 'Frustration',
       'Perceived Exertion (1)', 'Perceived Exertion (2)', 'mw_labels',
       'pw_labels', 'mwpw_labels'],
      dtype='object')

In [ ]:
subject_df

,Unnamed: 0,Participant ID,Mental Demand,Physical Demand,Temporal Demand,Performance,Effort,Frustration,Perceived Exertion (1),Perceived Exertion (2),mw_labels,pw_labels,mwpw_labels
6,6,1002,21,10,14,13,21,15,17,12,1.0,1.0,3.0
7,7,1002,21,13,13,14,21,19,18,13,0.0,1.0,2.0
8,8,1002,21,19,21,12,21,20,16,18,1.0,2.0,5.0
9,9,1002,21,20,20,14,21,20,17,17,0.0,2.0,4.0
10,10,1002,20,11,19,14,21,20,14,16,0.0,0.0,0.0
11,11,1002,21,16,21,12,21,21,17,18,1.0,0.0,1.0
